<a href="https://colab.research.google.com/github/madhurendra0089/Assignment/blob/main/AutoEncoder_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import random
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dropout, Conv3D, UpSampling3D, MaxPooling3D, Flatten, Dense, AveragePooling3D, BatchNormalization, Activation, Add, Reshape, UpSampling3D, Conv3DTranspose, Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.api._v2.keras import activations
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the preprocessed data
path = "/content/drive/MyDrive/MRI_ED/"

x_train = np.load(path+'x_train.npy')
x_test  = np.load(path+'x_test.npy')
x_val   = np.load(path+'x_val.npy')

y_train = np.load(path+'y_train.npy')
y_test  = np.load(path+'y_test.npy')
y_val   = np.load(path+'y_val.npy')

# Slicing Data->shuffle->batch->prefetch
batch_size=10
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train,)).shuffle(len(x_train)).batch(batch_size).prefetch(2)
validation_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val,)).shuffle(len(x_val)).batch(batch_size).prefetch(2)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1)

print("#x_train:" + str(x_train.shape)+" #y_train:" +str(y_train.shape)+" #x_val:"+ str(x_val.shape)+" #y_val:"+ str(y_val.shape)+" #x_test:"+str(x_test.shape)+" #y_test:"+ str(y_test.shape))

#x_train:(407, 128, 128, 64) #y_train:(407,) #x_val:(100, 128, 128, 64) #y_val:(100,) #x_test:(142, 128, 128, 64) #y_test:(142,)


# **Model**

In [ ]:
# 3D autoencoder

num_classes = 1
input_img = Input(shape=(128, 128, 64, 1))

# Encoder
x = Conv3D(16, 3, activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = MaxPooling3D(2)(x)

x = Conv3D(32, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling3D(2)(x)

x = Conv3D(64, 2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded = MaxPooling3D(2)(x)

# Decoder
x = Conv3DTranspose(64, 3, activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = UpSampling3D(2)(x)

x = Conv3DTranspose(32, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling3D(2)(x)

x = Conv3DTranspose(16, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling3D(2)(x)

decoded = Conv3D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

# Train autoencoder
autoencoder.fit(train_dataset, epochs=10, batch_size=32, validation_data=validation_dataset)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 64, 1   0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 128, 128, 64, 16   448       
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 128, 128, 64, 16   64        
 Normalization)              )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 64, 64, 32, 16)    0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 64, 64, 32, 32)    1385

In [ ]:
# Load
autoencoder = tf.keras.models.load_model(path+"my_autoencoder")


# Extract encoder
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('max_pooling3d_2').output)

# binary classification model using the extracted encoder
encoder_output = encoder.output
flattened = Flatten()(encoder_output)
x = Dropout(0.5)(flattened)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

classification_model = Model(inputs=autoencoder.input, outputs=output)
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classification_model.summary()

# Train binary classification model
classification_model.fit(x_train, y_train, epochs=50, batch_size=10, validation_data=(x_val, y_val))


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 64, 1   0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 128, 128, 64, 16   448       
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 128, 128, 64, 16   64        
 Normalization)              )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 64, 64, 32, 16)    0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 64, 64, 32, 32)    1385

In [ ]:
# Evaluating the model's performance
def performance(y_pred):
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))

In [ ]:
y_pred = classification_model.predict(x_test)
y_pred = [1 if num > 0.5 else 0 for num in y_pred]

performance(y_pred)

5/5 [==============================] - 5s 535ms/step
Confusion Matrix:
[[68  3]
 [58 13]]
Accuracy: 0.5704225352112676
Precision: 0.8125
Recall: 0.18309859154929578
F1 Score: 0.2988505747126437
